In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

In [2]:
# Function to infer naming conventions for left and right eyes
def infer_naming_conventions(directory):
    left_files = glob(os.path.join(directory, "left", "*.bmp"))
    right_files = glob(os.path.join(directory, "right", "*.bmp"))

    left_pattern = os.path.basename(left_files[0]).replace("1.bmp", "{}.bmp")
    right_pattern = os.path.basename(right_files[0]).replace("1.bmp", "{}.bmp")

    return left_pattern, right_pattern

# Function to load images from a given directory with inferred naming pattern
def load_images(directory, left_pattern, right_pattern, num_images):
    left_images = []
    right_images = []

    for i in range(1, num_images + 1):
        left_path = glob(os.path.join(directory, "left", left_pattern.format(i)))[0]
        right_path = glob(os.path.join(directory, "right", right_pattern.format(i)))[0]

        left_img = load_img(left_path, target_size=(128, 128))
        right_img = load_img(right_path, target_size=(128, 128))

        left_img_array = img_to_array(left_img)
        right_img_array = img_to_array(right_img)

        left_images.append(left_img_array)
        right_images.append(right_img_array)

    return np.array(left_images), np.array(right_images)

# Assuming you have 46 persons, repeat the process for each person and concatenate the data
all_left_images = []
all_right_images = []
labels = []

for person_id in range(1, 46):  # Assuming 46 persons
    person_directory = os.path.join("MMU-Iris-Database", str(person_id))

    left_pattern, right_pattern = infer_naming_conventions(person_directory)

    left_eye_images, right_eye_images = load_images(
        person_directory,
        left_pattern=left_pattern,
        right_pattern=right_pattern,
        num_images=5
    )

    # Create labels (person IDs)
    person_labels = np.full((10,), person_id - 1)  # Adjust index to start from 0

    all_left_images.append(left_eye_images)
    all_right_images.append(right_eye_images)
    labels.append(person_labels)

# Concatenate data for all persons
all_left_images = np.concatenate(all_left_images)
all_right_images = np.concatenate(all_right_images)
labels = np.concatenate(labels)

# Combine left and right eye images
all_images = np.concatenate([all_left_images, all_right_images], axis=0)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, labels, test_size=0.2, random_state=42)


In [3]:
def create_cnn_model(input_shape):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten layer
    model.add(layers.Flatten())

    # Dense (fully connected) layers
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization

    model.add(layers.Dense(46, activation='softmax'))  # Assuming 46 classes (persons)

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Assuming input shape is (128, 128, 3) for RGB images
input_shape = (128, 128, 3)

# Create the CNN model
cnn_model = create_cnn_model(input_shape)

# Display the model summary
cnn_model.summary()

# Train the model
history = cnn_model.fit(X_train, y_train, epochs=15, validation_split=0.2)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

In [4]:
def create_improved_cnn_model(input_shape):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    # Flatten layer
    model.add(layers.Flatten())

    # Dense (fully connected) layers
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(46, activation='softmax'))  # Assuming 46 classes (persons)

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Assuming input shape is (128, 128, 3) for RGB images
input_shape = (128, 128, 3)

# Create the improved CNN model
improved_cnn_model = create_improved_cnn_model(input_shape)

# Display the model summary
improved_cnn_model.summary()

# Train the model
history_improved = improved_cnn_model.fit(X_train, y_train, epochs=25, validation_split=0.2)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                      

In [7]:
def create_deeper_cnn_model(input_shape):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    # Flatten layer
    model.add(layers.Flatten())

    # Dense (fully connected) layers
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(46, activation='softmax'))  # Assuming 46 classes (persons)

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Assuming input shape is (128, 128, 3) for RGB images
input_shape = (128, 128, 3)

# Create the deeper CNN model
deeper_cnn_model = create_deeper_cnn_model(input_shape)

# Display the model summary
deeper_cnn_model.summary()

# Train the model
history_deeper = deeper_cnn_model.fit(X_train, y_train, epochs=50, validation_split=0.2)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 64, 64, 32)        0         
 ng2D)                                                           
                                                                 
 batch_normalization_14 (Ba  (None, 64, 64, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_15 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 32, 32, 64)        0         
 ng2D)                                                           
                                                      